https://www.freecodecamp.org/news/an-intro-to-advantage-actor-critic-methods-lets-play-sonic-the-hedgehog-86d6240171d/
https://towardsdatascience.com/understanding-actor-critic-methods-931b97b6df3f
http://rail.eecs.berkeley.edu/deeprlcourse-fa17/f17docs/lecture_5_actor_critic_pdf.pdf
https://danieltakeshi.github.io/2017/03/28/going-deeper-into-reinforcement-learning-fundamentals-of-policy-gradients/
https://danieltakeshi.github.io/2018/06/28/a2c-a3c/
https://danieltakeshi.github.io/2017/04/02/notes-on-the-generalized-advantage-estimation-paper/
https://github.com/ikostrikov/pytorch-a2c-ppo-acktr-gail/blob/master/a2c_ppo_acktr/algo/a2c_acktr.py
https://github.com/higgsfield/RL-Adventure-2

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions as distributions

import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import numpy as np
import gym
from utils import SubprocVecEnv

In [2]:
SEED = 1234

np.random.seed(SEED);
torch.manual_seed(SEED);

In [3]:
N_ENVS = 10

def make_env(env_name, seed):
    def _thunk():
        env = gym.make(env_name)
        env.seed(seed)
        return env
    return _thunk

env = [make_env('CartPole-v1', SEED+i) for i in range(N_ENVS)]
env = SubprocVecEnv(env)

assert isinstance(env.observation_space, gym.spaces.Box)
assert isinstance(env.action_space, gym.spaces.Discrete)

In [4]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout = 0.25):
        super().__init__()

        self.fc_1 = nn.Linear(input_dim, hidden_dim)
        self.fc_2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.fc_1(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.fc_2(x)
        return x

In [5]:
INPUT_DIM = env.observation_space.shape[0]
HIDDEN_DIM = 128
OUTPUT_DIM = env.action_space.n

actor = MLP(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
critic = MLP(INPUT_DIM, HIDDEN_DIM, 1)

In [6]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0)
        
actor.apply(init_weights)
critic.apply(init_weights)

MLP(
  (fc_1): Linear(in_features=4, out_features=128, bias=True)
  (fc_2): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.25)
)

In [7]:
LEARNING_RATE = 0.01

actor_optimizer = optim.Adam(actor.parameters(), lr = LEARNING_RATE)
critic_optimizer = optim.Adam(critic.parameters(), lr = LEARNING_RATE)

In [8]:
def train(env, actor, critic, actor_optimizer, critic_optimizer, n_steps, discount_factor):
    
    log_prob_actions = []
    values = []
    rewards = []
    done = False
    episode_reward = 0
    
    state = env.get_state()
    
    print(state)
    assert False
    
    for step in range(n_steps):

        state = torch.FloatTensor(state) #[n_envs, observation_space]

        action_preds = actor(state) #[n_envs, action_space]
        value_pred = critic(state) #[n_envs, 1]
                 
        action_probs = F.softmax(action_preds, dim = -1) #[n_envs, action_space]
                            
        dist = distributions.Categorical(action_probs)

        action = dist.sample() #[n_envs]
                
        log_prob_action = dist.log_prob(action) #[n_envs]
                
        #action now numpy array across all envs
        state, reward, _, _ = envs.step(action.cpu().numpy())
        
        reward = torch.FloatTensor(reward) #[n_envs]
         
        log_prob_actions.append(log_prob_action)
        values.append(value_pred)
        rewards.append(reward)
    
    log_prob_actions = torch.cat(log_prob_actions)
    values = torch.cat(values).squeeze(-1)
    rewards = torch.cat(rewards)
    
    print(log_prob_actions.shape)
    print(values.shape)
    print(rewards.shape)
    
    assert False
    
    returns = calculate_returns(rewards, discount_factor)
    advantages = calculate_advantages(returns, values)
    
    policy_loss, value_loss = update_policy(advantages, log_prob_actions, returns, values, actor_optimizer, critic_optimizer)

    return policy_loss, value_loss, episode_reward

In [9]:
def calculate_returns(rewards, discount_factor, normalize = True):
    
    returns = []
    R = 0
    
    for r in reversed(rewards):
        R = r + R * discount_factor
        returns.insert(0, R)
        
    returns = torch.tensor(returns)
    
    if normalize:
        
        returns = (returns - returns.mean()) / returns.std()
        
    return returns

In [10]:
def calculate_advantages(returns, values, normalize = True):
    
    advantages = []
    
    for r, v in zip(reversed(returns), reversed(values)):
        advantage = r - v
        advantages.insert(0, advantage)
        
    advantages = torch.tensor(advantages)
    
    if normalize:
        
        advantages = (advantages - advantages.mean()) / advantages.std()
        
    return advantages

In [11]:
def update_policy(advantages, log_prob_actions, returns, values, actor_optimizer, critic_optimizer):
        
    policy_loss = - (advantages * log_prob_actions).mean()
    
    value_loss = F.smooth_l1_loss(returns, values).mean()
        
    actor_optimizer.zero_grad()
    critic_optimizer.zero_grad()
    
    policy_loss.backward()
    value_loss.backward()
    
    actor_optimizer.step()
    critic_optimizer.step()
    
    return policy_loss.item(), value_loss.item()

In [12]:
MAX_STEPS = 100_000
N_UPDATE_STEPS =  25
DISCOUNT_FACTOR = 0.99
N_TRIALS = 25
REWARD_THRESHOLD = 475
PRINT_EVERY = 10

episode_rewards = []

_ = env.reset()

for step in range(0, MAX_STEPS, N_UPDATE_STEPS):
    
    policy_loss, value_loss, episode_reward = train(env, actor, critic, actor_optimizer, critic_optimizer, N_UPDATE_STEPS, DISCOUNT_FACTOR)
    
    episode_rewards.append(episode_reward)
    
    mean_trial_rewards = np.mean(episode_rewards[-N_TRIALS:])
    
    if episode % PRINT_EVERY == 0:
    
        print(f'| Episode: {episode:4} | Mean Rewards: {mean_trial_rewards:6.2f} |')
    
    if mean_trial_rewards >= REWARD_THRESHOLD:
        
        print(f'Reached reward threshold in {episode} episodes')
        
        break

[[-0.01315549 -0.04012824  0.04801855 -0.0044493 ]
 [ 0.02261076 -0.00212182  0.02805487 -0.04669543]
 [ 0.0357257   0.02193746  0.03064245  0.04975038]
 [-0.00139162 -0.01176114 -0.00531662 -0.01581865]
 [-0.00507147  0.00256169  0.04808907  0.00017638]
 [-0.00109669  0.0322358  -0.01694608 -0.01722023]
 [ 0.02837055 -0.03522862 -0.00189175  0.01407424]
 [-0.03104116 -0.02814798 -0.0300252  -0.04693883]
 [ 0.0114702  -0.03376913 -0.01478205  0.0010523 ]
 [ 0.04792483  0.03988038 -0.03678987 -0.03159145]]


AssertionError: 

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(episode_rewards)
plt.xlabel('Episode', fontsize=20)
plt.ylabel('Reward', fontsize=20)
plt.hlines(REWARD_THRESHOLD, 0, len(episode_rewards), color='r')
plt.grid()